In [2]:
from ultralytics import YOLO
from os import path, listdir, makedirs
from shutil import rmtree, copy
import numpy as np
import time
import random
import cv2

### Train the full classifer

In [3]:
# Constants
IMAGE_SIZE = 640 # x 480
PROJECT_NAME = "logs"
RECIPE_NAME = "all"
TRAIN_RUN = f"{RECIPE_NAME}_run-{time.strftime('%Y%m%d-%H%M%S')}"
DATA_PATH = "./../../datasets/full"

# Model
MODEL_PATH = "./models/pretrained/yolov8n-obb-dotav1.pt"
TEST_PATH = "./models/recipes/dataset_desc_test.yaml"
VAL_PATH = "./models/recipes/dataset_desc.yaml"
TRAIN_PARAM = {
    # Model definition
    'data': VAL_PATH,
    'resume': False,
    'device': '0',
    'pretrained': True,
    # Names
    'project' : PROJECT_NAME,
    'name': TRAIN_RUN,
    # Training Parameters
    'batch': -1,
    'imgsz': IMAGE_SIZE,
    'epochs': 50,
    'patience': 5,
    'cos_lr': True,
    # "lr0": 0.05,
    # Augmentation
    'hsv_h': 0.05, # Higher than default for resistor
    'hsv_s': 0.3, # Colours should not change too much
    'hsv_v': 0.2, # Colours should not change too much
    'degrees': 180, # Rotation
    'translate': 0.1, # Translation
    'scale': 0.8, # Scaling - camera is always at the same distance
    'shear': 10.0, # Shearing
    'perspective': 0.0, # Perspective
    'flipud': 0.5, # Flip up-down
    'fliplr': 0.5, # Flip left-right
    'mosaic': 0.5, # Mosaic
    'mixup': 0.0, # Mixup
    'copy_paste': 0.0, # Copy-paste
    'crop_fraction': 1.0, # Crop fraction
    # Loss weights
    # 'cls' : 1.0, # Class
    # 'box' : 4.0, # Box accuracy
    # 'dfl' : 1.5, # Help manage unbalanced classes
    # Post parameters
    'save': True,
    'save_period': 5,
    'plots': False,
    # Misc
    'verbose': False,
}

PATHS = {
    'train': f"{DATA_PATH}/current/images/train",
    'val': f"{DATA_PATH}/current/images/val",
    'test': f"{DATA_PATH}/current/images/test",
    'resistors' : f"{DATA_PATH}/resistor/imgs",
    'ceramic_cap' : f"{DATA_PATH}/ceramic_capacitor/imgs",
    "none" : "./datasets/none"
}


In [3]:
# Reorganise the data
PARAM = {
    'path' : DATA_PATH,
    'train' : 0.7,
    'val' : 0.2,
    'test' : 0.1,
    'include' : [
        'resistor',
        'capacitor',
        'ceramic_capacitor',
        'film_capacitor',
        'inductor',
        'led',
        'wire'
    ]
}
# Remove old dataset
DATASET_FOLDER = f"{DATA_PATH}/current"
if path.exists(DATASET_FOLDER):
    for folder in listdir(DATASET_FOLDER):
        rmtree(path.join(DATASET_FOLDER, folder), ignore_errors=True)
    # Make label folder
    makedirs(path.join(DATASET_FOLDER, 'labels'), exist_ok=True)
    # Make train, val, test folders
    for folder in ['train', 'val', 'test']:
        makedirs(path.join(DATASET_FOLDER, 'images', folder), exist_ok=True)
        makedirs(path.join(DATASET_FOLDER, 'labels', folder), exist_ok=True)

# Get all component images from all folders
basenames = []
# For component folder in dataset
for folder in listdir(PARAM['path']):
    # Skip if not in include
    if folder not in PARAM['include']: continue
    # For each subfolder in component folder
    imgfiles = listdir(path.join(PARAM['path'], folder, 'imgs'))
    labfiles = listdir(path.join(PARAM['path'], folder, 'labels'))
    bases = [path.join(folder, 'imgs', path.splitext(f)[0]) for f in imgfiles]
    basenames.extend(bases)
    print(f"Found {len(imgfiles)} images and {len(labfiles)} labels in {folder}")

# Split the data into train, val, test
random.shuffle(basenames)
train = int(len(basenames) * PARAM['train'])
val = int(len(basenames) * PARAM['val'])
test = int(len(basenames) * PARAM['test'])
print(f"Split into {train} train, {val} val, {test} test. Total: {train+val+test} images.")
train_set = basenames[:train]
val_set = basenames[train:train+val]
test_set = basenames[train+val:]

# Copy the images and labels to the new dataset folder
for folder, dataset in zip(['train', 'val', 'test'], [train_set, val_set, test_set]):
    for base in dataset:
        filename = path.split(base)[1]
        copy(path.join(PARAM['path'], f"{base}.png"), path.join(DATASET_FOLDER, 'images', folder, f"{filename}.png"))
        base = base.replace('imgs', 'labels')
        copy(path.join(PARAM['path'], f"{base}.txt"), path.join(DATASET_FOLDER, 'labels', folder, f"{filename}.txt"))


Found 164 images and 164 labels in capacitor
Found 264 images and 264 labels in ceramic_capacitor
Found 66 images and 66 labels in film_capacitor
Found 52 images and 52 labels in inductor
Found 96 images and 96 labels in led
Found 258 images and 259 labels in resistor
Found 75 images and 75 labels in wire
Split into 682 train, 195 val, 97 test. Total: 974 images.


In [86]:
# Tensorboard logging
%load_ext tensorboard
%tensorboard --logdir "logs" --port=6005

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [83]:
! taskkill /PID 9424 /F
! taskkill /IM "tensorboard.exe" /F

ERROR: The process "9424" not found.
ERROR: The process "tensorboard.exe" not found.


In [34]:
# Raw model
# model = YOLO("yolov8n-obb.yaml").to('cuda')

# Train
model = YOLO(MODEL_PATH).to('cuda')
model.train(**TRAIN_PARAM)

New https://pypi.org/project/ultralytics/8.2.32 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.31  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
engine\trainer: task=obb, mode=train, model=./models/pretrained/yolov8n-obb-dotav1.pt, data=./models/recipes/dataset_desc.yaml, epochs=50, time=None, patience=5, batch=-1, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=logs, name=all_run-20240614-0444198, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nm

train: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\datasets\full\current\labels\train... 682 images, 72 backgrounds, 0 corrupt: 100%|██████████| 754/754 [00:00<00:00, 1300.00it/s]


train: New cache created: C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\datasets\full\current\labels\train.cache


val: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\datasets\full\current\labels\val.cache... 195 images, 20 backgrounds, 0 corrupt: 100%|██████████| 215/215 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.000359375), 72 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to logs\all_run-20240614-0444198
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/50      6.52G      1.134      1.805      1.516         25        640: 100%|██████████| 17/17 [00:06<00:00,  2.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.23it/s]

                   all        215        195      0.807      0.853      0.899       0.68



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/50      6.52G      1.085       1.45      1.505         52        640:  35%|███▌      | 6/17 [00:02<00:03,  3.00it/s]

In [32]:
# Test the model
metrics = model.val(data=VAL_PATH)

Ultralytics YOLOv8.2.31  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)


val: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\datasets\full\current\labels\val... 195 images, 20 backgrounds, 0 corrupt: 100%|██████████| 215/215 [00:00<00:00, 1067.00it/s]


val: New cache created: C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\datasets\full\current\labels\val.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.89it/s]

                   all        215        195      0.833      0.948      0.984      0.759
Speed: 1.2ms preprocess, 2.9ms inference, 0.0ms loss, 2.4ms postprocess per image


In [33]:
confusion_matrix = metrics.confusion_matrix.matrix

remove = [4, 5, 6, 9]
filtered_matrix = np.delete(confusion_matrix, remove, axis=0)
filtered_matrix = np.delete(filtered_matrix, remove, axis=1)
print(confusion_matrix)
print(filtered_matrix)

[[          0           0           0           0           0           0           0           0           0           0           0           0]
 [          0           0           0           0           0           0           0           0           0           0           0           0]
 [          0           0           0           0           0           0           0           0           0           0           0           0]
 [          0           0           0           0           0           0           0           0           0           0           0           0]
 [          0           0           0           0           0           0           0           0           0           0           0           0]
 [          0           0           0           0           0           0           0           0           0           0           0           0]
 [          0           0           0           0           0           0           0           0           0         

In [23]:
metrics.confusion_matrix.matrix = filtered_matrix
metrics.confusion_matrix.nc = 7
metrics.confusion_matrix.plot(names=('resistor', 'capacitor', 'ceramic_capacitor', "inductor", "led", "wire", "film_capacitor"))

In [132]:
metrics.names

{0: "{0: 'resistor'}",
 1: "{1: 'capacitor'}",
 2: "{2: 'ceramic_cap'}",
 3: "{3: 'inductors'}",
 4: "{4: 'diodes'}",
 5: "{5: 'mosfet'}",
 6: "{6: 'transistor'}",
 7: "{7: 'leds'}",
 8: "{8: 'wire'}",
 9: "{9: 'ics'}",
 10: "{10: 'film_cap'}"}

### Grab component in box

In [15]:
def crop_image_to_bbox(image, bbox):
    """
    Crop the image to the region within the bounding box.

    Args:
    - image (np.array): The input image.
    - bbox (tensor): The coordinates of the bounding box in xyxyxyxy format.

    Returns:
    - cropped_image (np.array): The cropped image.
    """
    # Convert tensor to numpy array and ensure it's on the CPU
    bbox = bbox.cpu().numpy()[0]

    # Compute the width and height of the bounding box
    width = int(np.linalg.norm(bbox[0] - bbox[1]))
    height = int(np.linalg.norm(bbox[1] - bbox[2]))

    # Compute the center of the bounding box
    center = np.mean(bbox, axis=0).astype(int)

    # Compute the rotation angle of the bounding box
    angle = np.degrees(np.arctan2(bbox[1, 1] - bbox[0, 1], bbox[1, 0] - bbox[0, 0]))
    rotation_matrix = cv2.getRotationMatrix2D((int(center[0]), int(center[1])), angle, 1.0)

    # Apply the rotation to the image
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

    # Get the bounding box in the rotated image
    x, y = center - [width // 2, height // 2]
    cropped_image = rotated_image[y:y+height, x:x+width]

    return cropped_image


### Try model

In [18]:
DATA = PATHS['test']

try:
    model
    print("Loaded trained model")
except:
    # Load the best model
    # List dir and find latest run
    runs = listdir(f"./logs")
    runs.sort()
    while True:
        latest_run = runs.pop()
        if path.isdir(f"./logs/{latest_run}") and RECIPE_NAME in latest_run:
            break
    latest_run = "all_run-20240614-041748"
    modelPath = f"./logs/{latest_run}/weights/best.pt"
    print(f"Latest run: {latest_run}, loading model from {modelPath}")
    bestModel = YOLO(modelPath).to('cuda')
    model = bestModel
    print("Loaded best model")

while cv2.waitKey(0) != ord('q'):
    cv2.destroyAllWindows()
    randomFile = random.choice(listdir(DATA))
    # randomFile = "obb_resistor_yellow_violet_black_gold_brown_470E-1-1_30.png"
    print(f"Random file: {randomFile}")
    results = model.predict(f"{DATA}/{randomFile}", show=True)
    if len(results[0].obb.xyxyxyxy) > 0:
        cropped_img = crop_image_to_bbox(results[0].orig_img, results[0].obb.xyxyxyxy)
        cv2.imshow("Cropped Image", cropped_img)
cv2.destroyAllWindows()

Latest run: all_run-20240614-041748, loading model from ./logs/all_run-20240614-041748/weights/best.pt
Loaded best model
Random file: obb_inductor_473_19.png

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\..\..\datasets\full\current\images\test\obb_inductor_473_19.png: 480x640 11.0ms
Speed: 1.5ms preprocess, 11.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
[284  67]
Random file: obb_ceramic_capacitor_102_14.png

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\..\..\datasets\full\current\images\test\obb_ceramic_capacitor_102_14.png: 480x640 15.5ms
Speed: 182.5ms preprocess, 15.5ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
[357  78]
Random file: obb_led_red_2.png

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\..\..\datasets\full\current\images\test\obb_led_red_2.png: 480x640 11.5ms
Speed: 1.0ms pr

In [17]:
cv2.destroyAllWindows()

### Draw vertices to verify direction

In [33]:
import numpy as np
def draw_bounding_box(image, bbox, bbox_format, color=(0, 255, 0), thickness=2):
    """
    Draws a bounding box on the image with an arrow indicating orientation for the xyxyxyxy format.

    Args:
    - image (np.array): The input image.
    - bbox (tensor): The bounding box coordinates.
    - bbox_format (str): The format of the bounding box ('xywhr', 'xyxy', 'xyxyxyxy', 'xyxyxyxyn').
    - color (tuple): The color of the bounding box (default is green).
    - thickness (int): The thickness of the bounding box lines (default is 2).

    Returns:
    - image_with_box (np.array): The image with the bounding box drawn.
    """
    image_with_box = image.copy()  # Make a copy of the image to avoid overwriting

    if bbox_format == 'xywhr':
        # xywhr: [x_center, y_center, width, height, rotation (in radians)]
        x_center, y_center, width, height, rotation = bbox[0]
        center = (int(x_center.item()), int(y_center.item()))
        size = (int(width.item()), int(height.item()))
        angle = np.degrees(rotation.item())

        rect = ((center[0], center[1]), (size[0], size[1]), angle)
        box = cv2.boxPoints(rect)
        box = np.int0(box)

    elif bbox_format == 'xyxy':
        # xyxy: [x_min, y_min, x_max, y_max]
        x_min, y_min, x_max, y_max = bbox[0]
        box = np.array([[x_min.item(), y_min.item()],
                        [x_max.item(), y_min.item()],
                        [x_max.item(), y_max.item()],
                        [x_min.item(), y_max.item()]], dtype=np.int0)

    elif bbox_format == 'xyxyxyxy':
        # xyxyxyxy: [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        box = np.array(bbox[0].cpu(), dtype=np.int0)

    elif bbox_format == 'xyxyxyxyn':
        # xyxyxyxyn: normalized [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        h, w = image.shape[:2]
        box = np.array(bbox[0].cpu() * np.array([w, h]), dtype=np.int0)

    else:
        raise ValueError(f"Unsupported bbox_format: {bbox_format}")

    # Draw the polygon
    image_with_box = cv2.polylines(image_with_box, [box], isClosed=True, color=color, thickness=thickness)

    # Draw an arrow for the xyxyxyxy format to indicate orientation
    if bbox_format == 'xyxyxyxy':
        start_point = (int(box[0][0]), int(box[0][1]))
        end_point = (int(box[1][0]), int(box[1][1]))
        image_with_box = cv2.arrowedLine(image_with_box, start_point, end_point, color, thickness, tipLength=0.3)

     # Number each vertex
    for i, (x, y) in enumerate(box):
        cv2.putText(image_with_box, str(i+1), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), thickness, cv2.LINE_AA)

    return image_with_box

DATA = PATHS['resistors']
randomFile = random.choice(listdir(DATA))
print(f"Random file: {randomFile}")
results = model.predict(f"{DATA}/{randomFile}", show=True)

cv2.imshow(randomFile, draw_bounding_box(results[0].orig_img, results[0].obb.xyxyxyxy, 'xyxyxyxy'))
cv2.waitKey(0)
cv2.destroyAllWindows()

Random file: obb_resistor_orange_white_black_brown_brown_390E1-1_15.png

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\resistor\imgs\obb_resistor_orange_white_black_brown_brown_390E1-1_15.png: 480x640 53.0ms
Speed: 5.5ms preprocess, 53.0ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)


C:\Users\Shaheen\AppData\Local\Temp\ipykernel_30816\1325440353.py:39: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.array(bbox[0].cpu(), dtype=np.int0)


### Export model to ONNX

In [75]:
model.export(format="onnx", opset=13)


PyTorch: starting from 'logs\all_run-20240610-045510\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 16, 8400) (6.4 MB)

ONNX: starting export with onnx 1.14.1 opset 13...
ONNX: export success  1.0s, saved as 'logs\all_run-20240610-045510\weights\best.onnx' (11.9 MB)

Export complete (1.2s)
Results saved to C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\logs\all_run-20240610-045510\weights
Predict:         yolo predict task=obb model=logs\all_run-20240610-045510\weights\best.onnx imgsz=640  
Validate:        yolo val task=obb model=logs\all_run-20240610-045510\weights\best.onnx imgsz=640 data=./models/recipes/dataset_desc.yaml  
Visualize:       https://netron.app


'logs\\all_run-20240610-045510\\weights\\best.onnx'

In [79]:
import onnxruntime as ort
import cv2
import numpy as np
import random
from os import listdir


# Create the ONNX inference session
onnxModel = ort.InferenceSession(f"./logs\\all_run-20240610-045510\\weights\\best.onnx")
CONFIDENCE_THRESHOLD = 0.1

def preprocess(image):
    input_shape = onnxModel.get_inputs()[0].shape  # get input shape
    input_height, input_width = input_shape[2], input_shape[3]

    image = cv2.resize(image, (input_width, input_height))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # convert to RGB
    image = image.astype(np.float32)
    image = image / 255.0  # normalize to [0, 1]
    image = np.transpose(image, (2, 0, 1))  # convert to CHW
    image = np.expand_dims(image, axis=0)  # add batch dimension

    return image

def postprocess(output, original_image):
    # Debugging: print the shape of the output
    print(f"Output shape: {output.shape}")

    output = np.squeeze(output)  # remove batch dimension

    boxes = []
    for i in range(output.shape[1]):
        box = output[:, i]
        x_center, y_center, width, height, objectness = box[:5]
        if objectness > CONFIDENCE_THRESHOLD:
            x_center *= original_image.shape[1]  # scale back to original image size
            y_center *= original_image.shape[0]
            width *= original_image.shape[1]
            height *= original_image.shape[0]

            x_min = int(x_center - width / 2)
            y_min = int(y_center - height / 2)
            x_max = int(x_center + width / 2)
            y_max = int(y_center + height / 2)
            boxes.append((x_min, y_min, x_max, y_max))
    
    print(f"Filtered boxes: {len(boxes)}")

    for box in boxes:
        cv2.rectangle(original_image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

    return original_image

# Fetch the actual input name from the model
input_name = onnxModel.get_inputs()[0].name

while cv2.waitKey(0) != ord('q'):
    cv2.destroyAllWindows()
    randomFile = random.choice(listdir(DATA))
    print(f"Random file: {randomFile}")

    image = cv2.imread(f"{DATA}/{randomFile}")
    input_tensor = preprocess(image)

    # Perform inference
    results = onnxModel.run(None, {input_name: input_tensor})[0]
    output_image = postprocess(results, image.copy())

    cv2.imshow(randomFile, output_image)

cv2.destroyAllWindows()

Random file: obb_ceramic_capacitor_472k_1.png
Output shape: (1, 16, 8400)
Filtered boxes: 0
Random file: obb_resistor_green_brown_black_red_brown_510E2-1_1.png
Output shape: (1, 16, 8400)
Filtered boxes: 10
Random file: obb_resistor_orange_blue_black_gold_brown_gold_360E-1-1_14.png
Output shape: (1, 16, 8400)
Filtered boxes: 12
Random file: obb_ceramic_capacitor_10_6.png
Output shape: (1, 16, 8400)
Filtered boxes: 0
Random file: obb_led_yellow_4.png
Output shape: (1, 16, 8400)
Filtered boxes: 0


In [70]:
print(results)

[[[     5.2338      17.409      26.986 ...      510.89       548.1      579.53]
  [     5.7821       5.787      7.7035 ...      599.64      588.09      583.31]
  [     11.832       24.92      39.459 ...      238.35      186.63      154.18]
  ...
  [ 3.8743e-07  5.6624e-07  5.6624e-07 ...    4.53e-06  4.2915e-06  4.4405e-06]
  [  1.809e-05  1.0133e-05  7.6294e-06 ...  1.4275e-05  1.7732e-05  3.8892e-05]
  [  0.0060101    0.029564    0.015276 ...  -0.0082752  -0.0093197    0.017739]]]


In [64]:
cv2.destroyAllWindows()

In [49]:
while cv2.waitKey(0) != ord('q'):
    cv2.destroyAllWindows()
    randomFile = random.choice(listdir(DATA))
    randomFile = "obb_resistor_yellow_violet_black_gold_brown_470E-1-1_30.png"
    print(f"Random file: {randomFile}")
    results = model.predict(f"{DATA}/{randomFile}", show=True)
    cropped_img = crop_image_to_bbox(results[0].orig_img, results[0].obb.xyxyxyxy)
    cv2.imshow("Cropped Image", cropped_img)
cv2.destroyAllWindows()

Random file: obb_resistor_yellow_violet_black_gold_brown_470E-1-1_30.png

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\current\images\test\obb_resistor_yellow_violet_black_gold_brown_470E-1-1_30.png: 480x640 47.5ms
Speed: 3.5ms preprocess, 47.5ms inference, 7.5ms postprocess per image at shape (1, 3, 480, 640)
[377 153]
